In [1]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.io as pio
import plotly.graph_objs as go

In [2]:
# https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/world.geojson

with open('../data/world_geojson.txt','r') as f :
    world_geojson = json.load(f)

print(f"Number of countries : {len(world_geojson['features'])}")

Number of countries : 177


In [3]:
# Turn it into a dataframe and retrieve the countries and their associated 3-letter id
feat_df = pd.DataFrame.from_dict(world_geojson['features'])
feat_df['country'] = feat_df['properties'].apply(lambda x : x['name'])
feat_df['country'] = feat_df['country'].astype('category')
feat_df = feat_df[['id','country']]
feat_df.head()

,id,country
0,AFG,Afghanistan
1,AGO,Angola
2,ALB,Albania
3,ARE,United Arab Emirates
4,ARG,Argentina


In [4]:
# Generate random color for the map
test_df = feat_df.copy()
test_df['color_test'] = pd.Series(np.random.randint(12, size=len(test_df)))

In [5]:
# Function that plots a world map
def world_fig(df, loc='id', value=None, cont_color=None, disc_color_map=None,
              range_color=None, label='X') :
    fig = px.choropleth(df, geojson=world_geojson, locations=loc, color=value,
                            color_continuous_scale=cont_color,
                            color_discrete_map=disc_color_map,
                            range_color=range_color,
                            hover_name="country", hover_data={value:True},
                            labels={value:label})
    
    fig.update_geos(bgcolor='white',framecolor='white',framewidth=0)
    fig.update_traces(hovertemplate='Country : %{hovertext} <br>Value : %{customdata[0]}')
    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, width=1100, height=500, dragmode=False, margin_autoexpand=True)

    fig.show()


In [6]:
nd_world = pd.read_excel('../data/emdat_public_2021_12_11_query_uid-ucCmQ2.xlsx', header=6)
nd_world.head(1)

C:\Users\rapha\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1960-0013-CHL,1960,13,NaN,Natural,Geophysical,Earthquake,Tsunami,NaN,NaN,...,NaN,NaN,NaN,550000.0,4813056.0,11.427251,NaN,NaN,NaN,NaN


In [7]:
nd_world['Disaster Type'].unique()
nd_types = ['Flood', 'Storm', 'Earthquake', 'Drought', 'Extreme temperature ', 'Volcanic activity', 'Wildfire']

nd_world['code'] = nd_world.apply(lambda d : d['Dis No'][-3:], axis=1)

In [8]:
nd_disaster = nd_world.groupby(['Disaster Type','code'])
nd_tmp = nd_world[(nd_world['code'] == 'CHE') & (nd_world['Disaster Type'] == 'Flood')]
len(nd_tmp)

8

In [9]:
nd_world_count = pd.DataFrame(len(nd_types)*list(feat_df['country'].unique()),columns=['country'])
nd_world_count['Disaster'] = nd_world_count.apply(lambda x : nd_types[int((len(nd_types)*x.name)/len(nd_world_count))], axis=1)
nd_world_count['id'] = nd_world_count.apply(lambda c : feat_df[feat_df['country']==c['country']]['id'].values[0], axis=1)

nd_world_count['Tot'] = nd_world_count.apply(lambda T : len(nd_world[(nd_world['code'] == T['id']) & (nd_world['Disaster Type'] == T['Disaster'])]), axis=1)
nd_world_count[nd_world_count['country']=='Switzerland'].head(10)

,country,Disaster,id,Tot
28,Switzerland,Flood,CHE,8
205,Switzerland,Storm,CHE,33
382,Switzerland,Earthquake,CHE,0
559,Switzerland,Drought,CHE,0
736,Switzerland,Extreme temperature,CHE,6
913,Switzerland,Volcanic activity,CHE,0
1090,Switzerland,Wildfire,CHE,0


In [11]:
scales = ['Blues','Oranges','Greens','BuGn','Purples','Reds','Blues']
label_types = ['Flood', 'Storm', 'Earthquake', 'Drought', 'Heat', 'Volcano', 'Wildfire']

data_slider = []
for n,group in enumerate(nd_types) :
    df_segmented =  nd_world_count[(nd_world_count['Disaster']== group)].copy()
    clr = scales[nd_types.index(group)]
    for col in df_segmented.columns:
        df_segmented[col] = df_segmented[col].astype(str)

    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['country'],
                        z=df_segmented['Tot'].astype(float),
                        locationmode='country names',
                        colorscale = clr,
                        hovertemplate="Country : %{location}<br>"+group+" : %{z}<extra></extra>")
    data_slider.append(data_each_yr)

steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label=label_types[i])
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=6, font=dict(size=10), steps=steps)]

layout = dict(geo=dict(scope='world'),
              sliders=sliders)

fig = go.Figure(data=data_slider, layout=layout)
fig.update_geos(framewidth=0, projection_type="natural earth", center=dict(lon=20, lat=19))
#fig.update_layout(width=1100, height=500, dragmode=False)
fig.update_layout(
        autosize=False,
        margin = dict(l=40, r=40, b=20, t=10, pad=4, autoexpand=True), width=650, height=400)
fig.show()
#fig.write_html('slider_disasters.html')